# Image pixel-level saliency using VGG-16 Conv-net

Lets have plots appear inline:

In [ ]:
%matplotlib inline

We're going to need `numpy`, `matplotlib`, `skimage`, `torch`, `torch.nn`, `torch.nn.functional` and `torchvision`.

Our `utils` module provides an image preparation function.

In [ ]:
import os, time
import numpy as np
from matplotlib import pyplot as plt
import torch, torch.nn as nn, torch.nn.functional as F
import torchvision
import skimage.transform, skimage.util
from skimage.util import montage

import utils
import imagenet_classes

torch_device = torch.device('cuda:0')

## Load an image to process

In [ ]:
def load_image_block(path, x_pos=None, y_pos=None, width=None, height=None):
    return plt.imread(path)[y_pos:y_pos+height, x_pos:x_pos+width]

def load_image(path):
    return plt.imread(path)

# Extract a 896 x 896 block surrounding the peacock
peacock = load_image_block(os.path.join('images', 'P1013781.JPG'), 652, 1800, 896, 896)
plt.imshow(peacock)
plt.show()

## Load the model

To save some time, we are going to use the `VGG16Model` class from `pretrained_vgg_models` to build the network and load its parameters:

In [ ]:
# Build it, requesting that the pre-trained model weights are loaded
# The call to the `to` method moves it onto the GPU
vgg16_net = torchvision.models.vgg.vgg16(pretrained=True).to(torch_device)

# Call the eval() method; we are not training
vgg16_net.eval()

# Also, set a variable 
MODEL_MEAN = np.array([0.485, 0.456, 0.406])
MODEL_STD = np.array([0.229, 0.224, 0.225])

## Discover the pixel-level image saliency
We will be using guided backpropagation as described in [1], referring to [2], implemented as a [custom PyTorch autograd function](https://pytorch.org/tutorials/beginner/examples_autograd/two_layer_net_custom_function.html):

[1]: Springenberg et al. (2015): "Striving for Simplicity - The All Convolutional Net", http://arxiv.org/abs/1412.6806
[2]: Utku Ozbulak, "Pytorch implementation of convolutional neural network visualization techniques", http://github.com/utkuozbulak/pytorch-cnn-visualizations

Please see the above references for more information on the technique and its implementation.

In [ ]:
class GuidedBackpropReLUFunction (torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input = grad_input.clamp(min=0) * (input >= 0).float()
        return grad_input
    
class GuidedBackpropReLU (nn.Module):
    def forward(self, x):
        return GuidedBackpropReLUFunction.apply(x)

    
def apply_guided_backprop(model):
    if isinstance(model, (nn.ModuleList, nn.Sequential)):
        for i, sub in enumerate(model):
            if isinstance(sub, nn.ReLU):
                model[i] = GuidedBackpropReLU()
            else:
                apply_guided_backprop(sub)
    else:
        for name, sub in model.named_children():
            if isinstance(sub, nn.ReLU):
                setattr(model, name, GuidedBackpropReLU())
            else:
                apply_guided_backprop(sub)
            
apply_guided_backprop(vgg16_net)
        



## Define a function to visualise saliency

Define a function to predict the class of the image, compute a saliency map and visualise it (for convenience):

In [ ]:
def f_classify_images(image_batch):
    """
    image_batch - images as a NumPy 4D array - (sample, channel, height, width)
    return: probabilities - (sample, class)
    """
    # We don't need gradients here as we are only performing inference/prediction
    with torch.no_grad():
        x = torch.tensor(image_batch, dtype=torch.float, device=torch_device)
        logits = vgg16_net(x)
        probs = F.softmax(logits, dim=1)
        return probs.detach().cpu().numpy()
    
def f_class_saliency_map(image, model, class_index=None):
    x = torch.tensor(image, dtype=torch.float, device=torch_device, requires_grad=True)
    logits = vgg16_net(x)
    probs = F.softmax(logits, dim=1)
    if class_index is not None:
        cls_prob = probs[:,class_index]
    else:
        cls_prob, _ = torch.max(probs, dim=1)
    cls_prob.sum().backward()
    return x.grad.detach().cpu().numpy()

def visualise_cls_saliency(img, saliency_class=None):
    # Prepare the image for use with VGG
    raw_img, img_for_vgg = utils.prepare_image(img, MODEL_MEAN, MODEL_STD, image_size=224)
    
    # Predict the image class
    pred_prob = f_classify_images(img_for_vgg)
    pred_cls = np.argmax(pred_prob, axis=1)[0]
    pred_confidence = pred_prob[0,pred_cls]
    
    # Print what we found:
    print('Predicted class = {}, name = {}, confidence = {:.2%}'.format(
            pred_cls, imagenet_classes.IMAGENET_CLASSES[pred_cls], pred_confidence))
    
    # If no saliency class was specified, use the predicted one
    if saliency_class is None:
        saliency_class = pred_cls
    
    # Compute a saliency image as a tensor
    # Choose the function depending on whether the class is specified or not
    saliency = f_class_saliency_map(img_for_vgg, saliency_class)
    
    # Extract sample 0, move channel axis to the back
    saliency_image = saliency[0].transpose(1, 2, 0)

    # Plot
    plt.figure(figsize=(10,10))
    plt.subplot(2,2,1)
    plt.imshow(np.sqrt(np.maximum(saliency_image, 0.0) / saliency_image.max()))

    plt.subplot(2,2,2)
    plt.imshow(np.sqrt(np.maximum(-saliency_image, 0.0) / -saliency_image.min()))

    plt.subplot(2,2,3)
    plt.imshow(raw_img)
    plt.imshow(np.sqrt(np.maximum(saliency_image, 0.0) / saliency_image.max()), alpha=0.6)

    plt.subplot(2,2,4)
    plt.imshow(raw_img)
    plt.imshow(np.sqrt(np.maximum(-saliency_image, 0.0) / -saliency_image.min()), alpha=0.6)
    plt.show()

## Lets try it out:

Try it on the peacock:

In [ ]:
visualise_cls_saliency(peacock, 84)

In [ ]:
flowers = load_image_block(os.path.join('images', 'P8131065.JPG'), 600, 1300, 224*3, 224*3)
plt.imshow(flowers)

visualise_cls_saliency(flowers)

In [ ]:
dog = load_image_block(os.path.join('images', 'P8131065.JPG'), 1000, 900, 224*4, 224*4)
plt.imshow(dog)

visualise_cls_saliency(dog)

In [ ]:
wall_and_flowers = load_image_block(os.path.join('images', 'P8131065.JPG'), 0, 500, 224*3, 224*3)
plt.imshow(wall_and_flowers)

visualise_cls_saliency(wall_and_flowers)

In [ ]:
wall_and_flowers = load_image_block(os.path.join('images', 'P8131065.JPG'), 900, 0, 224*3, 224*3)
plt.imshow(wall_and_flowers)

visualise_cls_saliency(wall_and_flowers)

### Visualising saliency at lower levels of the network

It is often informative to visualise pixel level saliency with respect to activations of the convolutional layers of the network.

In the following examples, we visualise saliency with respect to the output at the pooling layers that follow the convolutional layers. It allows us to see how the lower levels of the network detect small simple features - such as colour and gradient - while higher levels detect more complex features.

First, we define functions similar to those before for generating and plotting the saliency images.

In [ ]:
# Cache for storing compiled functions; compiling Theano functions can be slow
_SALIENCY_FUNCTIONS_BY_LAYER = {}

def f_saliency_map(image, model, channel_index):
    x = torch.tensor(image, dtype=torch.float, device=torch_device, requires_grad=True)
    y = model(x)
    chn_out = y[:, channel_index]
    chn_out.max().backward()
    return x.grad.detach().cpu().numpy()

        
def f_model(image, model):
    x = torch.tensor(image, dtype=torch.float, device=torch_device)
    y = model(x)
    return y.detach().cpu().numpy()

        
def visualise_low_level_saliency(img, model, n_channels, patch_shape, max_cols=8, out_path=None):
    half_patch_shape = patch_shape[0] // 2, patch_shape[1] // 2
    # Prepare the image for use with VGG
    raw_img, img_for_vgg = utils.prepare_image(img, MODEL_MEAN, MODEL_STD, image_size=224)
    
    y = f_model(img_for_vgg, model)
    
    # Compute the downsampling factor at the specified layer of the network
    scale_factor = raw_img.shape[0] // y.shape[2]
    
    # Get the maximum activation value in each channel
    chn_max = y.max(axis=(0,2,3))
    # Get the channel indices in order of decreasing maximum activation value
    channel_indices = np.argsort(chn_max[::-1])
    channel_indices = list(channel_indices[:n_channels])
    
    saliency_patches = []
    for chn in channel_indices:
        # Get the activation values in the selected channel
        y_chn = y[0, chn, :, :]
        # Get the position of the maximum activation
        y_chn_max_pos = np.unravel_index(np.argmax(y_chn.flatten()), y_chn.shape)
        # Apply scale factor
        y_chn_max_pos = y_chn_max_pos[0] * scale_factor, y_chn_max_pos[1] * scale_factor
        # Clamp position so that the patch lies completely in the bounds of the image
        y_chn_max_pos = (min(max(y_chn_max_pos[0], half_patch_shape[0]), raw_img.shape[0]-half_patch_shape[0]-1),
                         min(max(y_chn_max_pos[1], half_patch_shape[1]), raw_img.shape[1]-half_patch_shape[1]-1))
        y_sal = f_saliency_map(img_for_vgg, model, chn)

        # Extract sample 0, move channel axis to the back
        y_sal = y_sal[0].transpose(1, 2, 0)

        # Compute patch bounds
        p_lower = y_chn_max_pos[0] - half_patch_shape[0], y_chn_max_pos[1] - half_patch_shape[1]
        p_upper = y_chn_max_pos[0] + half_patch_shape[0], y_chn_max_pos[1] + half_patch_shape[1]
        # Extract patches
        img_patch = raw_img[p_lower[0]:p_upper[0], p_lower[1]:p_upper[1], :]
        sal_patch = y_sal[p_lower[0]:p_upper[0], p_lower[1]:p_upper[1], :]
        # Scale saliency to [0,1] range, then apply sqrt to make the result more visible when displayed
        sal_patch = np.sqrt(np.maximum(sal_patch, 0.0) / sal_patch.max())
        # Store
        saliency_patches.append((img_patch, sal_patch))
        
    N = len(saliency_patches) * 2
    num_cols = max(1024 // patch_shape[1], 2)
    if num_cols % 2 == 1:
        num_cols -= 1
    num_cols = min(num_cols, max_cols)
    
    num_rows = n_channels * 2 // num_cols
    if num_rows * num_cols < N:
        num_rows += 1
    
    patch_width = 16.0 / num_cols
    height = float(num_rows * patch_shape[0] * patch_width) / patch_shape[1]

    # Plot
    patches = []
    for ip, sp in saliency_patches:
        patches.append(ip[None, :, :, :])
        patches.append(sp[None, :, :, :])
    patches = np.concatenate(patches, axis=0)
    m = montage(patches, grid_shape=(num_rows, num_cols), multichannel=True)
    
    plt.figure(figsize=(16,int(height+0.5)))
    plt.imshow(m)
    plt.show()
    
    if out_path is not None:
        plt.imsave(out_path, m)

Lets get the layout of the `features` component of the model so we can select the parts of the model up to just after the pooling layers:

In [ ]:
print(vgg16_net.features)

In [ ]:
net_to_pool1 = vgg16_net.features[:5]
net_to_pool2 = vgg16_net.features[:10]
net_to_pool3 = vgg16_net.features[:17]
net_to_pool4 = vgg16_net.features[:24]
net_to_pool5 = vgg16_net.features[:]

#### Try it out

In [ ]:
for model, layer_name, patch_size, n_chns in zip([net_to_pool1, net_to_pool2, net_to_pool3, net_to_pool4, net_to_pool5],
                                                 ['pool1', 'pool2', 'pool3', 'pool4', 'pool5'],
                                                 [32, 32, 48, 96, 192], [16, 16, 12, 8, 8]):
    channels = range(11)
    print('----------------------------------------------------')
    print('PIXEL LEVEL SALIENCY OF PEACOCK IMAGE AT LAYER {}'.format(layer_name))
    print('----------------------------------------------------')
    visualise_low_level_saliency(peacock, model, n_chns, (patch_size,patch_size))

In [ ]:
for model, layer_name, patch_size, n_chns in zip([net_to_pool1, net_to_pool2, net_to_pool3, net_to_pool4, net_to_pool5],
                                                 ['pool1', 'pool2', 'pool3', 'pool4', 'pool5'],
                                                 [32, 32, 48, 96, 192], [16, 16, 12, 8, 8]):
    channels = range(11)
    print('----------------------------------------------------')
    print('PIXEL LEVEL SALIENCY OF FLOWERS IMAGE AT LAYER {}'.format(layer_name))
    print('----------------------------------------------------')
    visualise_low_level_saliency(flowers, model, n_chns, (patch_size,patch_size))

In [ ]:
for model, layer_name, patch_size, n_chns in zip([net_to_pool1, net_to_pool2, net_to_pool3, net_to_pool4, net_to_pool5],
                                                 ['pool1', 'pool2', 'pool3', 'pool4', 'pool5'],
                                                 [32, 32, 48, 96, 192], [16, 16, 12, 8, 8]):
    channels = range(11)
    print('----------------------------------------------------')
    print('PIXEL LEVEL SALIENCY OF DOG IMAGE AT LAYER {}'.format(layer_name))
    print('----------------------------------------------------')
    visualise_low_level_saliency(dog, model, n_chns, (patch_size,patch_size))


In [ ]:
for model, layer_name, patch_size, n_chns in zip([net_to_pool1, net_to_pool2, net_to_pool3, net_to_pool4, net_to_pool5],
                                                 ['pool1', 'pool2', 'pool3', 'pool4', 'pool5'],
                                                 [32, 32, 48, 96, 192], [16, 16, 12, 8, 8]):
    channels = range(11)
    print('----------------------------------------------------')
    print('PIXEL LEVEL SALIENCY OF WALL AND FLOWERS IMAGE AT LAYER {}'.format(layer_name))
    print('----------------------------------------------------')
    visualise_low_level_saliency(wall_and_flowers, model, n_chns, (patch_size,patch_size))
